In [66]:

import functools
import jax
import os

from datetime import datetime
from jax import numpy as jp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output


import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac
import pprint

In [67]:
params = model.load_params("./runs/run_reacher_vis_test_s_1/ckpt/final")

In [68]:
processor_params, policy_params, encoders_params = params
sa_encoder_params, g_encoder_params = encoders_params['sa_encoder'], encoders_params['g_encoder']

In [69]:
from crl_new import networks
from utils import get_env_config
import pickle

args_path = "./runs/run_reacher_vis_test_s_1/args.pkl"

with open(args_path, "rb") as f:
    args = pickle.load(f)

config = get_env_config(args)
args


Namespace(exp_name='reacher_vis_test', num_timesteps=2000, max_replay_size=2000, min_replay_size=0, num_evals=5, episode_length=1001, action_repeat=4, discounting=0.997, num_envs=4, batch_size=32, seed=1, unroll_length=200, multiplier_num_sgd_steps=1, env_name='reacher', normalize_observations=False, use_tested_args=False, log_wandb=False, policy_lr=0.0006, alpha_lr=0.0003, critic_lr=0.0003, contrastive_loss_fn='binary', logsumexp_penalty=0.0, random_goals=0.0, use_old_trans_actor=False, use_old_trans_alpha=False, disable_entropy_actor=False, use_traj_idx_wrapper=False, sgd_to_env=0.15640625)

In [70]:
from utils import create_env
env = create_env(args)
obs_size = env.observation_size
action_size = env.action_size
print(obs_size)
print(action_size)

13
2


In [71]:
crl_networks = networks.make_crl_networks(config, obs_size, action_size)

In [72]:
crl_networks.policy_network

FeedForwardNetwork(init=<function make_policy_network.<locals>.<lambda> at 0x7f266b9af400>, apply=<function make_policy_network.<locals>.apply at 0x7f266b9af9a0>)

In [73]:
inference_fn = networks.make_inference_fn(crl_networks)
inference_fn = inference_fn(params[:2])
inference_fn

<function crl_new.networks.make_inference_fn.<locals>.make_policy.<locals>.policy(observations: jax.Array, key_sample: jax.Array) -> Tuple[jax.Array, Mapping[str, Any]]>

In [75]:
sa_encoder = lambda obs: crl_networks.sa_encoder.apply(processor_params, sa_encoder_params, obs)
g_encoder = lambda obs: crl_networks.g_encoder.apply(processor_params, g_encoder_params, obs)

g_encoder(jp.zeros(3))

Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)